In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from scipy import stats
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    #data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = cy.utils.ensure_dir('/data/user/ssclafani/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks')
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository(local_root='/data/user/ssclafani/data/analyses')
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, 
        cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get bkg fits for each template

In [ ]:
'''bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict:
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
    
    else:
        print('Running background trials for template {}'.format(key))
        bkg_dict[key] = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
'''

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    res_dict[key] = np.load(f_path)

In [ ]:
tr_dict

In [ ]:
tr = tr_dict['pi0']

In [ ]:
#Get Trial
trial = tr_dict['pi0'].get_one_trial(TRUTH=True)
L = tr_dict['pi0'].get_one_llh_from_trial(trial)
L.fit(**tr_dict['pi0'].fitter_args)

In [ ]:
ts_true, ns_true = tr.get_one_fit(TRUTH=True)
gamma_true = 2.7
flux_true = tr.to_E2dNdE(ns_true, E0=100, unit=1e3)

In [ ]:
print('From the TrialRunner:')
print(cy.inspect.get_llh_model(tr, -1))
print(cy.inspect.get_pdf_ratio_model(tr, -1))
print(cy.inspect.get_space_model(tr, -1))
print(cy.inspect.get_energy_model(tr, -1))

In [ ]:
space_eval = cy.inspect.get_space_eval(L, -1, 0) # 0: background events (1 would be for signal events)
energy_eval = cy.inspect.get_energy_eval(L, -1, 0)
print(space_eval)
print(energy_eval)

In [ ]:
L =tr.get_one_llh()
trial = tr.get_one_trial()


In [ ]:
'''gammas = np.r_[1:4.001:1/128.]
nss = np.linspace(0,2000,100)
GAMMA, NS = np.meshgrid(gammas, nss)
tss = L.scan_ts(nss, gamma=gammas)'''

In [ ]:
#Get Trial
trial = tr_dict['pi0'].get_one_trial(TRUTH=True)
L = tr_dict['pi0'].get_one_llh_from_trial(trial)
L.fit(**tr_dict['pi0'].fitter_args)

In [ ]:
dNdE = np.linspace (0, 10e-18,6000)
gammas = np.r_[1.0:3.5001:2**-6]

get_ns_from_flux = lambda dNdE,  **params: cy.hyp.PowerLawFlux(params['gamma']).to_ns (dNdE, L.get_acc_total (**params), E0=1e5, unit=1, E2dNdE=False)
    
mesh_tss, (mesh_fluxs, mesh_gammas) = L.scan_ts (dNdE, gamma=gammas, get_ns_from_arg=get_ns_from_flux)

In [ ]:
Z = np.max(mesh_tss) - mesh_tss

x, y =np.unravel_index(np.argmin(Z), Z.shape)
#print(stats.norm.isf(np.mean(bkg_dict['pi0'] > np.max(mesh_tss))))
flux_best = dNdE[x]
gamma_best = gammas[y]
print(gamma_best, flux_best)

fig, ax = plt.subplots ()
pc = ax.pcolormesh (gammas, 1e18*dNdE , Z, cmap='viridis_r', vmin=0, vmax = 10.01)
contour_fracs = [1 - 2*stats.norm.sf(1), 0.95]
dLLHs = stats.chi2.ppf (contour_fracs, 2)
print(dLLHs)
contour_labels = {dLLH: r'{:.0f}$\%$'.format (100*frac) for (dLLH, frac) in zip (dLLHs, contour_fracs)}
cont = ax.contour (gammas, 1e18*dNdE, Z, dLLHs, colors='w')
ax.clabel (cont, inline=True, fmt=contour_labels)
ax.plot (gamma_true, flux_true, '.', ms=10, color='r', label='Analysis')
cb = fig.colorbar (pc)
cb.set_label (r'$-2\Delta\ln\mathcal{L}$')
ax.plot (gamma_best,1e18* flux_best, '.', ms=10, color='w', label='Best Fit')

#cb.set_ticks (np.r_[0:10000.1:100])
ax.set_xlabel (r'$\gamma$')
ax.set_ylabel (r'dN/dE at 100TeV [GeV$^{-1}$  s$^{-1}$ cm$^{-2}$ ] ')
ax.set_xlim (1.5, 3.5)
#ax.set_ylim (0, .3e-11)
plt.legend()
ax.grid();

In [ ]:
'''repo = cy.selections.Repository(local_root='/data/user/ssclafani/data/analyses')
mesespecs = cy.selections.MESEDataSpecs.MESC_2010_2016
ana_mese = cy.get_analysis(
    repo, 'version-001-p01', mesespecs, 
    #gammas=np.r_[0.1:6.01:0.125],
)'''

In [ ]:
'''pi0_map = cy.selections.mrichman_repo.get_template ('Fermi-LAT_pi0_map')
conf = {
    # desired template
    'template': pi0_map,
    # desired baseline spectrum
    'flux':     cy.hyp.PowerLawFlux(2.5),
    # desired fixed spectrum in likelihood
    'fitter_args': dict(gamma=2.5),
    # use signal subtracting likelihood
    'sigsub': True,
    # fastest construction:
    # weight from acceptance parameterization rather
    # than pixel-by-pixel directly from MC
    'fast_weight': True,
    # cache angular-resolution-smeared maps to disk
    'dir': cy.utils.ensure_dir('{}/templates/pi0'.format(ana_dir))
}


mese_tr = cy.get_trial_runner(ana=ana_mese, conf=conf)'''

In [ ]:
#Lm =mese_tr.get_one_llh(TRUTH=True,)
#MTST  = Lm.fit(gamma=2.5)

In [ ]:
#MTST

In [ ]:
'''dNdE = np.logspace (-20, -17, 2000)
gammas = np.r_[1.0:3.5001:2**-5]

get_ns_from_flux = lambda dNdE,  **params: cy.hyp.PowerLawFlux(params['gamma']).to_ns (dNdE, Lm.get_acc_total (**params), E0=1e5, unit=1, E2dNdE=False)
    
mesh_tss_m, (mesh_fluxs_m, mesh_gammas_m) = Lm.scan_ts (dNdE, gamma=gammas, get_ns_from_arg=get_ns_from_flux)'''

In [ ]:
#MTST[0]

In [ ]:
#Z = MTST[0] - mesh_tss_m

In [ ]:
'''Z = np.max(mesh_tss_m)- mesh_tss_m

x, y =np.unravel_index(np.argmin(Z), Z.shape)
#print(stats.norm.isf(np.mean(bkg_dict['pi0'] > np.max(mesh_tss))))
flux_best_m = dNdE[x]
gamma_best_m = gammas[y]
print(gamma_best, flux_best)

fig, ax = plt.subplots ()
pc = ax.pcolormesh (gammas, 1e18*dNdE , Z, cmap='viridis_r', vmin=0, vmax = 10.01)
contour_fracs = [1 - 2*stats.norm.sf(1), 0.95]
dLLHs = stats.chi2.ppf (contour_fracs, 2)
print(dLLHs)
contour_labels = {dLLH: r'{:.0f}$\%$'.format (100*frac) for (dLLH, frac) in zip (dLLHs, contour_fracs)}
cont_m = ax.contour (gammas, 1e18*dNdE, Z, dLLHs, colors='w')
ax.clabel (cont, inline=True, fmt=contour_labels)
#ax.plot (gamma_true, flux_true, '.', ms=10, color='r', label='Analysis')
cb = fig.colorbar (pc)
cb.set_label (r'$-2\Delta\ln\mathcal{L}$')
ax.plot (gamma_best_m ,1e18*flux_best_m, '.', ms=10, color='w', label='Best Fit')

#cb.set_ticks (np.r_[0:10000.1:100])
ax.set_xlabel (r'$\gamma$')
ax.set_ylabel (r'10$^{18} \cdot$ dN/dE at 100TeV [GeV$^{-1}$ s$^{-1}$ cm$^{-2}$ ] ')
ax.set_xlim (1.0, 3.5)
ax.set_ylim (.15, 10)
#plt.legend()
ax.grid();'''

In [ ]:
tracks_95 = [
    1.36331, 0.11138,
    1.41100, 0.22737,
1.47402, 0.49818,
1.50685, 0.64512,
1.55248, 0.83431,
1.58068, 1.02092,
1.60432, 1.27361,
1.63732, 1.51276,
1.65325, 1.69047,
1.67610, 1.97166,
1.69461, 2.15143,
1.71475, 2.44769,
1.73006, 2.62075,
1.74932, 2.90552,
1.76158, 3.07892,
1.78171, 3.39074,
1.79506, 3.55182,
1.81082, 3.84713,
1.82460, 4.02710,
1.84233, 4.31843,
1.85612, 4.49641,
1.87231, 4.80045,
1.88763, 4.97845,
1.90338, 5.26203,
1.91717, 5.44034,
1.93533, 5.73643,
1.95065, 5.91231,
1.97079, 6.21281,
1.98610, 6.38640,
2.00856, 6.67355,
2.02550, 6.84457,
    
2.05356, 7.13233,
2.07473, 7.30434,
2.11216, 7.57913,
2.14564, 7.70339,
2.21063, 7.75919,
2.24805, 7.67124,

2.31699, 7.25502,
2.33813, 7.02663,
2.36331, 6.79419,
2.37050, 6.57627,
2.39209, 6.32930,
2.40647, 6.08232,
2.43885, 5.84988,
2.45324, 5.63196,
2.47122, 5.35593,
2.48549, 5.18492,
2.50016, 4.98800,
2.50637, 4.73253,
2.52606, 4.49458,
2.55629, 4.20210,
2.55801, 4.00333,
2.57106, 3.71091,
2.57697, 3.54307,
2.60011, 3.36389,
2.61871, 3.03148,
2.63669, 2.79903,
2.65468, 2.50847,
2.66906, 2.31961,
2.69065, 2.07264,
2.70144, 1.85472,
2.72880, 1.61472,
2.73622, 1.43394,
2.75580, 1.29402,
2.76723, 1.06771,
2.76998, 0.90268,
2.78349, 0.66221,
2.79137, 0.43099,
2.73381, 0.28571,
2.68245, 0.21584,
2.63605, 0.20088,
2.55755, 0.18402,
2.50360, 0.18402,
2.40647, 0.12591,


]

In [ ]:
tracks_68 = [
1.82775, 1.72507,
1.83813, 1.95642,
1.84892, 2.15981,
1.86331, 2.42131,
1.87770, 2.66828,
1.89157, 2.88165,
1.90338, 3.07654,
1.91689, 3.28847,
1.92899, 3.48619,
1.94803, 3.69036,
1.96050, 3.87199,
1.97626, 4.05494,
1.99398, 4.23710,
2.01368, 4.43437,
2.03534, 4.62766,
2.06095, 4.80209,

2.09640, 4.97006,
2.13973, 5.04310,
2.18306, 4.95498,

2.20932, 4.74259,
2.23741, 4.51332,
2.25889, 4.24783,
2.26906, 4.01384,
2.27760, 3.80437,
2.28088, 3.56839,
2.28548, 3.40135,
2.28285, 3.19983,
2.27872, 2.99870,
2.27169, 2.80343,
2.25959, 2.61461,
2.24258, 2.41057,
2.21457, 2.22825,
2.18503, 2.05439,
2.14958, 1.88655,
2.10822, 1.73259,
2.06489, 1.60460,
2.02156, 1.50264,
1.97823, 1.42237,
1.93490, 1.36886,
1.89157, 1.35295,
1.84824, 1.45274,
1.82775, 1.72507,

]

In [ ]:
mesc_68 = [
1.02134, 0.16030,
1.10913, 0.17853,
1.15204, 0.21909,
1.19561, 0.26337,
1.23917, 0.30983,
1.28274, 0.37226,
1.32630, 0.43856,
1.36987, 0.53486,
1.41344, 0.58204,
1.45700, 0.67060,
1.50057, 0.78384,
1.54414, 0.87914,
1.58770, 0.98636,
1.63259, 1.11931,
1.67483, 1.31084,
1.71840, 1.45252,
1.76221, 1.60582,
1.80388, 1.78791,
1.83722, 1.96922,
1.87621, 2.13440,
1.90119, 2.35488,
1.92930, 2.53462,
1.96395, 2.69366,
2.00009, 2.88359,
2.02421, 3.10004,
2.04713, 3.29697,
2.06891, 3.47527,
2.09380, 3.65863,
2.11965, 3.91793,
2.15010, 4.18039,
2.18723, 4.40780,
2.21347, 4.61577,
2.23808, 4.78335,
2.26839, 4.95472,

2.31248, 5.12323,    

2.35605, 5.14791,
2.39962, 5.05136,

2.43200, 4.87062,
2.46571, 4.69495,
2.49436, 4.50393,
2.51663, 4.32983,
2.54022, 4.14999,
2.56398, 3.96101,
2.58756, 3.77379,
2.60754, 3.59544,
2.62735, 3.42776,
2.64913, 3.24903,
2.66418, 3.14110,
2.86666, 1.62939,
2.89157, 1.46281,
2.92241, 1.29591,
2.95409, 1.12374,
2.99007, 0.92977,
2.96767, 0.82233,
2.92241, 0.79618,
2.87884, 0.79763,
2.83528, 0.79836,
2.79171, 0.79836,
2.74814, 0.79255,
2.70458, 0.78965,
2.66101, 0.77585,
2.61745, 0.76279,
2.57388, 0.74464,
2.53031, 0.72069,
2.48675, 0.69673,
2.44318, 0.66915,
2.39962, 0.63866,
2.35605, 0.60672,
2.31248, 0.57478,
2.26892, 0.54066,
2.22535, 0.50727,
2.18178, 0.47461,
2.13822, 0.44267,
2.09465, 0.41073,
2.05109, 0.37879,
2.00752, 0.34975,
1.96395, 0.32072,
1.92039, 0.29459,
1.87682, 0.26991,
1.83326, 0.24523,
1.78969, 0.22272,
1.74612, 0.20385,
1.70256, 0.18425,
1.63165, 0.15013,
1.55984, 0.15013,
1.48803, 0.13405,
1.40027, 0.10188,
]

In [ ]:
mesc_95=[
    1.04907, 0.16828,
1.00532, 0.16622,
1.09263, 0.20530,
1.13620, 0.26918,
1.17976, 0.31273,
1.22333, 0.33669,
1.26690, 0.39258,
1.31046, 0.43396,
1.35403, 0.50679,
1.39759, 0.59132,
1.44116, 0.66189,
1.48473, 0.76351,
1.52829, 0.85691,
1.57186, 0.98346,
1.61543, 1.08145,
1.64909, 1.21655,
1.68275, 1.36746,
1.72038, 1.49620,
1.75801, 1.67378,
1.80035, 1.78485,
1.83326, 1.96272,
1.87003, 2.14939,
1.89464, 2.36714,
1.93044, 2.56139,
1.95423, 2.77714,
1.98489, 2.93259,
2.00480, 3.13841,
2.03326, 3.28962,
2.05232, 3.55063,
2.08871, 3.73005,
2.09776, 3.93981,
2.11446, 4.10141,
2.13426, 4.34661,
2.15406, 4.51130,
2.17584, 4.80674,
2.19367, 4.95503,
2.21479, 5.24125,
2.23129, 5.40859,
2.25277, 5.70964,
2.27090, 5.87471,

2.29420, 6.17276,
2.31446, 6.33783,


2.34725, 6.62550,
2.37783, 6.76701,

    2.43922, 6.76084,

 2.47447, 6.62967,
2.51051, 6.35294,
2.53427, 6.20328,
2.56565, 5.91601,
2.58576, 5.76580,
    
2.61481, 5.46881,
2.63329, 5.32043,
2.66254, 5.01004,
2.68478, 4.85784,
2.71052, 4.56653,
2.73230, 4.41149,
2.76003, 4.12518,
2.95213, 2.65952,
2.97442, 2.50019,
3.02623, 2.18943,
3.05443, 2.05597,
3.10063, 1.82041,
3.13232, 1.68903,
3.18381, 1.46327,
3.22341, 1.35108,
3.27490, 1.14316,
3.31648, 1.04748,
3.37391, 0.87966,

]

In [ ]:
len(dNdE)
len(Z)
len(gammas)

In [ ]:
Z = ts_true - mesh_tss

x, y =np.unravel_index(np.argmin(Z), Z.shape)
#print(stats.norm.isf(np.mean(bkg_dict['pi0'] > np.max(mesh_tss))))
flux_best = dNdE[x]
gamma_best = gammas[y]
print(gamma_best, flux_best)

fig, ax = plt.subplots (figsize=(6,5))
pc = ax.pcolormesh (gammas, 1e18*dNdE , Z, cmap='viridis_r', vmin=0, vmax = 10.01)
contour_fracs = [1 - 2*stats.norm.sf(1), 0.95]
dLLHs = stats.chi2.ppf (contour_fracs, 2)
print(dLLHs)
contour_labels = {dLLH: r'{:.0f}$\%$'.format (100*frac) for (dLLH, frac) in zip (dLLHs, contour_fracs)}

cont = ax.contour (gammas, 1e18*dNdE, Z, dLLHs, colors='w', linestyles=['solid', 'dashed', 'dotted'], linewidths=3,zorder=10)

#ax.contour(cont_m, colors=['0.5', '0.5'], linestyles=['solid', 'dashed'], alpha=0.8, linewidths=2)
#ax.clabel (cont_m, inline=True, fmt=contour_labels)
ax.clabel (cont, inline=True, fmt=contour_labels,  fontsize=10, inline_spacing=5, manual=[(2.6,4),(2.7,2)])
#ax.plot (gamma_true, flux_true, '.', ms=10, color='r', label='Analysis')
cb = fig.colorbar (pc)
cb.set_label (r'$-2\Delta\ln\mathcal{L}$')
#ax.plot (gamma_best_m, 1e18*flux_best_m, '.', ms=10, color='0.5',alpha=0.5 )

ax.plot(np.nan, np.nan, c='w', ls='--', label='This Work')
ax.plot (gamma_best, 1e18*flux_best, '.', ms=10, color='w', label='Best Fit')


ax.plot(np.nan, np.nan, c='0.5', ls='--', alpha=0.8, label='IceCubeCascades (2019)')

#ax.plot(tracks_95[::2], tracks_95[1::2], c='0.3', alpha=0.8, ls='--', lw=2, label='IceCube Tracks (2016)')
#ax.plot(tracks_68[::2], tracks_68[1::2], c='0.3', alpha=0.8, ls='-', lw=2)
#ax.plot(2.07194, 3.11864, '.', ms=10, c='0.3', alpha=0.8)


ax.plot(mesc_68[::2], mesc_68[1::2], c='0.5', alpha=0.8, ls='-', lw=2)
ax.plot(mesc_95[::2], mesc_95[1::2], c='0.5', alpha=0.8, ls='--', lw=2)
ax.plot(2.02, 2.03, '.', ms=10, c='0.5', alpha=0.8)



#cb.set_ticks (np.r_[0:10000.1:100])
ax.set_xlabel (r'$\gamma$')
ax.set_ylabel (r'10$^{-18} \cdot$ dN/dE at 100TeV [GeV$^{-1}$ s$^{-1}$ cm$^{-2}$ ] ')
ax.set_xlim (1.0, 3.4)
ax.set_ylim (.15, 10)
plt.legend(framealpha=0.1, ncol = 2,labelcolor='w' )
#ax.grid();
plt.savefig('spectral_scan.pdf')

In [ ]:
dNdE_log = np.logspace (-19, -17, 2000)
gammas_log = np.r_[1.0:3.5001:2**-5]

get_ns_from_flux = lambda E2dNdE,  **params: cy.hyp.PowerLawFlux(params['gamma']).to_ns (dNdE_log, L.get_acc_total (**params),E0=1e5, unit=1, E2dNdE=False)
    
mesh_tss_log, (mesh_fluxs_log, mesh_gammas_log) = L.scan_ts (dNdE_log, gamma=gammas_log, get_ns_from_arg=get_ns_from_flux)

In [ ]:
Z

In [ ]:
Z = np.max(mesh_tss_log)- mesh_tss_log

x, y =np.unravel_index(np.argmin(Z), Z.shape)
#print(stats.norm.isf(np.mean(bkg_dict['pi0'] > np.max(mesh_tss))))
#flux_best = dNdE[x]
#gamma_best = gammas[y]
#print(gamma_best, flux_best)

fig, ax = plt.subplots ()
pc = ax.pcolormesh (gammas_log, dNdE_log , Z, cmap='viridis_r', vmin=0, vmax = 10.01)
contour_fracs = [1 - 2*stats.norm.sf(1), 0.95]
dLLHs = stats.chi2.ppf (contour_fracs, 2)
print(dLLHs)
contour_labels = {dLLH: r'{:.0f}$\%$'.format (100*frac) for (dLLH, frac) in zip (dLLHs, contour_fracs)}
cont = ax.contour (gammas_log,dNdE_log, Z, dLLHs, colors='w', linestyles=['solid', 'dashed', 'dotted'], linewidths=3,zorder=10)

#plt.contour(cont_m, colors=['C1', 'C1'])
ax.clabel (cont, inline=True, fmt=contour_labels, fontsize=10, inline_spacing=1)
#ax.plot (gamma_true, flux_true, '.', ms=10, color='r', label='Analysis')
cb = fig.colorbar (pc)
cb.set_label (r'$-2\Delta\ln\mathcal{L}$')
#ax.plot (gamma_best, flux_best, '.', ms=10, color='w', label='Best Fit')
ax.plot(mesc_68[::2], 1e-18*np.array(mesc_68[1::2]), c='0.5', alpha=0.8, ls='-', lw=2)
ax.plot(mesc_95[::2], 1e-18*np.array(mesc_95[1::2]), c='0.5', alpha=0.8, ls='--', lw=2)
ax.plot(2.02, 2.03e-18, '.', ms=10, c='0.5', alpha=0.8)
#cb.set_ticks (np.r_[0:10000.1:100])
ax.set_xlabel (r'$\gamma$')
ax.set_ylabel (r'dN/dE at 100TeV [GeV$^{-1}$ s$^{-1}$ cm$^{-2}$ ] ')
#ax.set_xlim (1.0, 3.5)
#ax.set_ylim (0, .3e-11)
#plt.legend()
ax.grid();
ax.set_yscale('log')

In [ ]:
np.array(mesc_68[1::2])